## Set Library Path

In [1]:
.libPaths("/home/vhaghani/anaconda3/envs/jupyter_nb_R4.4.2/lib/R/library")

## Load Libraries

In [3]:
library(readxl)
library(ggplot2)
library(viridis)
library(dplyr)
library(glue)
library(tidyr)

## Set Modules and GO Databases

In [4]:
# Read in data
module_membership <- read.table("06_WGCNA/filtered_module_membership.txt", header = TRUE, sep = "\t")

# Covnert to df
module_membership <- as.data.frame(module_membership)

# View
colnames(module_membership)

[1] "Probe"              "treatment"          "Module"            
 [4] "entrez_gene_id"     "external_gene_name" "greenyellow"       
 [7] "white"              "black"              "turquoise"         
[10] "plum1"              "skyblue3"           "paleturquoise"     
[13] "darkred"            "lightyellow"        "orangered4"        
[16] "royalblue"          "blue"               "brown"             
[19] "ivory"              "darkorange"         "midnightblue"      
[22] "darkgreen"          "salmon"             "purple"            
[25] "darkmagenta"        "grey60"             "skyblue"           
[28] "orange"             "darkolivegreen"     "steelblue"         
[31] "violet"             "floralwhite"        "lightgreen"        
[34] "darkturquoise"

The module removal was run after viewing the module trait Pearson correlation heatmap to remove modules related to litter effects and sex only, but unrelated to conditional differences and subset to only significant modules.

In [5]:
# Remove columns that don't correspond to modules
module_membership$Probe <- NULL
module_membership$treatment <- NULL
module_membership$Module <- NULL
module_membership$entrez_gene_id <- NULL
module_membership$external_gene_name <- NULL

# Subset to modules of interest
modules <- colnames(module_membership)

# View
print(modules)

 [1] "greenyellow"    "white"          "black"          "turquoise"     
 [5] "plum1"          "skyblue3"       "paleturquoise"  "darkred"       
 [9] "lightyellow"    "orangered4"     "royalblue"      "blue"          
[13] "brown"          "ivory"          "darkorange"     "midnightblue"  
[17] "darkgreen"      "salmon"         "purple"         "darkmagenta"   
[21] "grey60"         "skyblue"        "orange"         "darkolivegreen"
[25] "steelblue"      "violet"         "floralwhite"    "lightgreen"    
[29] "darkturquoise" 


## Make Dot Plots

In [ ]:
# List of GO databases
databases <- c("GO_Biological_Process_2023", "GO_Cellular_Component_2023", "GO_Molecular_Function_2023",
               "KEGG_2019_Mouse", "Panther_2016", "Reactome_2016", "RNAseq_DiseaseGene_DrugSigs_GEO")

In [ ]:
# Store GO data into data frame

# Initialize an empty dataframe
all_data <- data.frame()

# Iterate over each module and read in the corresponding Excel file
for (module in modules) {
  file_path <- glue("06_WGCNA/{module}_enrichr_results.xlsx")
  
  for (database in databases) {
    try({
      # Read the data from the Excel file
      df <- read_excel(file_path, sheet = database)
      
      if (nrow(df) == 0) {
        next  # Skip to the next database if no data
      }
      
      # Select the required columns and add module and database information
      df <- df %>%
        select(Term, Adjusted.P.value, Odds.Ratio) %>%
        mutate(Module = module, Database = database)
      
      # Append to the dataframe
      all_data <- bind_rows(all_data, df)
      
    }, silent = TRUE)
  }
}

# Check if any data was read
if (nrow(all_data) == 0) {
  stop("No data read from any of the Excel files.")
}

# Display the combined dataframe
head(all_data)

In [ ]:
# Filter out rows where the Adjusted.P.value is less than 0.1
all_data <- all_data %>%
  filter(Adjusted.P.value <= 0.05)

# Display the filtered dataframe
head(all_data)

In [ ]:
# Count the number of total terms and unique terms for each database
database_term_counts <- all_data %>%
  group_by(Database) %>%
  summarise(
    Total_Terms = n(),
    Unique_Terms = n_distinct(Term)
  )

# Print the table
print(database_term_counts)

In [ ]:
# Calculate the number of modules each term appears in for each database
term_module_counts <- all_data %>%
  group_by(Database, Term) %>%
  summarise(ModuleCount = n_distinct(Module), .groups = 'drop')

# Rank the terms within each database by the number of modules they appear in
ranked_terms <- term_module_counts %>%
  arrange(Database, desc(ModuleCount)) %>%
  group_by(Database) %>%
  slice_head(n = 25) %>%
  ungroup()

# Merge with the original data to filter the top 25 terms per database
filtered_data_top_25 <- all_data %>%
  semi_join(ranked_terms, by = c("Database", "Term"))

# Print the filtered data
head(filtered_data_top_25)

In [ ]:
# Count the number of total terms and unique terms for each database
filtered_database_term_counts <- filtered_data_top_25 %>%
  group_by(Database) %>%
  summarise(
    Total_Terms = n(),
    Unique_Terms = n_distinct(Term)
  )

# Print the table
print(filtered_database_term_counts)

In [ ]:
# Create plots for each database
for (database in unique(filtered_data_top_25$Database)) {
  database_filtered_data <- filtered_data_top_25 %>%
    filter(Database == database)
  
  if (nrow(database_filtered_data) > 0) {
    dot_plot <- ggplot(database_filtered_data, aes(x = Module, y = Term, size = Odds.Ratio, fill = Adjusted.P.value)) +
      geom_point(shape = 21) +
      scale_fill_viridis() +
      xlab('') + ylab('') +
      labs(
        title = 'Top Enrichr Terms Across Modules',
        subtitle = glue('{database}')
      ) +
      theme(
        panel.background = element_rect(fill = "white", color = NA),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.line = element_line(color = "black"),
        plot.background = element_rect(fill = "white", color = NA),
        axis.text.x = element_text(angle = 90, hjust = 1)
      )
    
    # Save the dot plot with the database name in the filename
    ggsave(filename = glue("06_WGCNA/top_25_dot_plot_{database}.pdf"), plot = dot_plot, height = 7, width = 15)
  } else {
    cat(glue("No data available for {database}. Skipping...\n"))
  }
}

# Combined Dot Plot

In [6]:
# List of GO databases
databases <- c("GO_Biological_Process_2023", "GO_Cellular_Component_2023", "GO_Molecular_Function_2023",
               "KEGG_2019_Mouse", "Reactome_2016")

In [7]:
# Create abbreviation mapping
database_abbreviations <- c(
  "GO_Biological_Process_2023" = "GO BP",
  "GO_Cellular_Component_2023" = "GO CC", 
  "GO_Molecular_Function_2023" = "GO MF",
  "KEGG_2019_Mouse" = "KEGG_Mouse",
  "Reactome_2016" = "REACTOME"
)

In [8]:
# Initialize an empty dataframe for all GO data (overrides previous one to save memory)
all_data <- data.frame()

In [9]:
# Store GO data into data frame

# Initialize an empty dataframe
all_data <- data.frame()

# Iterate over each module and read in the corresponding Excel file
for (module in modules) {
  file_path <- glue("06_WGCNA/{module}_enrichr_results.xlsx")
  
  for (database in databases) {
    try({
      # Read the data from the Excel file
      df <- read_excel(file_path, sheet = database)
      
      if (nrow(df) == 0) {
        next  # Skip to the next database if no data
      }
      
      # Select the required columns and add module and database information
      df <- df %>%
        filter(Adjusted.P.value <= 0.05) %>%
        select(Term, Adjusted.P.value, Odds.Ratio) %>%
        mutate(Module = module, Database = database) %>%
        arrange(Adjusted.P.value) %>%  
        slice_head(n = 3) 
      
      # Append to the dataframe
      all_data <- bind_rows(all_data, df)
      
    }, silent = TRUE)
  }
}

# Check if any data was read
if (nrow(all_data) == 0) {
  stop("No data read from any of the Excel files.")
}

# Display the combined dataframe
head(all_data)

,Term,Adjusted.P.value,Odds.Ratio,Module,Database
,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,Cytoplasmic Translation (GO:0002181),8.207676e-07,8.903484,greenyellow,GO_Biological_Process_2023
2,Macromolecule Biosynthetic Process (GO:0009059),1.084755e-05,5.280709,greenyellow,GO_Biological_Process_2023
3,Peptide Biosynthetic Process (GO:0043043),2.029099e-05,5.515487,greenyellow,GO_Biological_Process_2023
4,Cytosolic Large Ribosomal Subunit (GO:0022625),3.021899e-04,8.847450,greenyellow,GO_Cellular_Component_2023
5,Large Ribosomal Subunit (GO:0015934),3.021899e-04,8.847450,greenyellow,GO_Cellular_Component_2023
6,Cytosolic Small Ribosomal Subunit (GO:0022627),2.492542e-03,8.669292,greenyellow,GO_Cellular_Component_2023


In [11]:
# Generate dot plot - alternative approach
if (nrow(all_data) > 0) {
  
  # Create database abbreviations and assign colors
  all_data <- all_data %>%
    mutate(
      DB_Abbrev = ifelse(Database %in% names(database_abbreviations), 
                         database_abbreviations[Database], Database),
      Module = factor(Module),
      # Create simple term label with database
      Simple_Term = paste(DB_Abbrev, Term, sep = ": ")
    )
  
  # Create a color mapping for databases
  unique_dbs <- unique(all_data$DB_Abbrev)
  db_colors <- setNames(RColorBrewer::brewer.pal(max(3, length(unique_dbs)), "Set3")[1:length(unique_dbs)], 
                       unique_dbs)
  
  # Sort terms by database first, then by term name
  all_data <- all_data %>%
    arrange(DB_Abbrev, Term)
  
  # Ensure terms are ordered properly
  term_order <- unique(all_data$Simple_Term)
  all_data$Simple_Term <- factor(all_data$Simple_Term, levels = term_order)
  
  dot_plot <- ggplot(all_data, aes(x = Module, y = Simple_Term, size = Odds.Ratio, fill = Adjusted.P.value)) +
    geom_point(shape = 21, alpha = 0.8) +
    scale_fill_viridis(name = 'Adjusted P-value', direction = -1) +
    scale_size_continuous(name = 'Odds Ratio', range = c(1, 8)) +
    xlab('') + ylab('') +
    labs(title = 'Top Most Significant GO Terms Across Modules') +
    theme_minimal() +
    theme(
      panel.background = element_rect(fill = "white", color = NA),
      panel.grid.major = element_line(color = "grey90"),
      panel.grid.minor = element_blank(),
      axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5),
      axis.text.y = element_text(size = 10),
      legend.position = "right"
    )
  
  # Save the plot
  ggsave(filename = "06_WGCNA/combined_top_terms_dot_plot.pdf", 
         plot = dot_plot, height = 17, width = 16)
}

In [12]:
# Save data
write.csv(all_data, "06_WGCNA/combined_top_terms_dot_plot.csv", row.names = FALSE)